## Install dependencies

In [1]:
!pip install -q keras-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 944.9/944.9 kB 14.5 MB/s eta 0:00:00


## Set the backend

In this tutorial we will use Tensorflow as our backend, you can change it to "jax" or "torch" for JAX and PyTorch separately.

In [1]:
import os

# Try oicking a different value from "tensoroflow", "jax" and "torch".
os.environ["KERAS_BACKEND"] = "tensorflow"

In [2]:
import tensorflow as tf
import keras_core as keras

import numpy as np

Using TensorFlow backend


## Use a Keras layer

In [ ]:
layer = keras.layers.Dense(
    2,
    activation="relu",
    bias_initializer="zeros",
)
data = np.random.uniform(size=[3, 3])
layer(data)

Array([[0.31549454, 0.0904167 ],
       [0.46515635, 0.0611807 ],
       [0.53176594, 0.        ]], dtype=float32)

## Write your custom layer

In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.w = self.add_weight(
            shape=[input_dim, self.units], initializer="glorot_uniform",
        )
        self.b = self.add_weight(
            shape=[self.units,], initializer="zeros"
        )

    def call(self, inputs):
        return keras.ops.matmul(inputs, self.w) + self.b

layer = Linear(2)
layer(data)

Array([[ 0.22234565, -0.4042182 ],
       [ 0.5558392 , -1.1450278 ],
       [ 0.33656308, -0.83847624]], dtype=float32)

# Different ways of building the model

- Sequential API
- [**Preferred**] Functional API
- Subclassing model

In [ ]:
# Sequential model
sequential_model = keras.Sequential([
    keras.Input([28, 28, 3]),
    keras.layers.Conv2D(8, 2),
    keras.layers.MaxPool2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(2),
    keras.layers.Softmax(),
])

print(sequential_model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 27, 27, 8)         │        104 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 8)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten (Flatten)               │ (None, 1352)              │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_1 (Dense)                 │ (None, 2)                 │      2,706 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ softmax (Softmax)               │ (None, 2)                 │          0 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 2,810 (10.98 KB)

 Trainable params: 2,810 (10.98 KB)

 Non-trainable params: 0 (0.00 B)

None


In [4]:
# Functional model
inputs = keras.Input([28, 28, 3])
x = keras.layers.Conv2D(8, 2)(inputs)
x = keras.layers.MaxPool2D(2)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(2)(x)
outputs = keras.layers.Softmax()(x)

functional_model = keras.Model(inputs=inputs, outputs=outputs)
print(functional_model.summary())

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 3)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d (Conv2D)                 │ (None, 27, 27, 8)         │        104 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 8)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten (Flatten)               │ (None, 1352)              │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, 2)                 │      2,706 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ softmax (Softmax)               │ (None, 2)                 │          0 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 2,810 (10.98 KB)

 Trainable params: 2,810 (10.98 KB)

 Non-trainable params: 0 (0.00 B)

None


Let's build a complex model with functional api.

In [ ]:
# Multi-input model.
input_1 = keras.Input([28, 28, 3])
input_2 = keras.Input([28, 28, 3])
x_1 = keras.layers.Conv2D(8, 2)(input_1)
x_2 = keras.layers.Conv2D(8, 2)(input_2)
x = keras.layers.MaxPool2D(2)(x_1 + x_2)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(2)(x)
outputs = keras.layers.Softmax()(x)

multi_input_functional_model = keras.Model(inputs=inputs, outputs=outputs)
print(multi_input_functional_model.summary())

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 28, 28, 3) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ input_layer_3       │ (None, 28, 28, 3) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 27, 27, 8) │     104 │ input_layer_2[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 27, 27, 8) │     104 │ input_layer_3[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add (Add)           │ (None, 27, 27, 8) │       0 │ conv2d_2[0][0],      │
│                     │                   │         │ conv2d_3[0][0]       │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ max_pooling2d_2     │ (None, 13, 13, 8) │       0 │ add[0][0]            │
│ (MaxPooling2D)      │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ flatten_2 (Flatten) │ (None, 1352)      │       0 │ max_pooling2d_2[0][… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_3 (Dense)     │ (None, 2)         │   2,706 │ flatten_2[0][0]      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ input_layer_1       │ (None, 28, 28, 3) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ softmax_2 (Softmax) │ (None, 2)         │       0 │ dense_3[0][0]        │
└─────────────────────┴───────────────────┴─────────┴──────────────────────┘

 Total params: 2,914 (11.38 KB)

 Trainable params: 2,914 (11.38 KB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
# Subclassing `keras.Model`

class MyModel(keras.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._conv = keras.layers.Conv2D(8, 2)
        self._max_pool = keras.layers.MaxPool2D(2)
        self._flatten = keras.layers.Flatten()
        self._dense = keras.layers.Dense(2)
        self._softmax = keras.layers.Softmax()

    def call(self, inputs):
        x = self._conv(inputs)
        x = self._max_pool(x)
        x = self._flatten(x)
        x = self._dense(x)
        return self._softmax(x)

subclass_model = MyModel()
subclass_model(np.random.uniform(size=[1, 28, 28, 3]))
subclass_model.summary()

Model: "my_model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ ?                         │        104 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ ?                         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten_4 (Flatten)             │ ?                         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_5 (Dense)                 │ ?                         │      2,706 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ softmax_4 (Softmax)             │ ?                         │          0 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 2,810 (10.98 KB)

 Trainable params: 2,810 (10.98 KB)

 Non-trainable params: 0 (0.00 B)

# Models are callable.

`keras.Model` instances are also callable. You can call models as if it is a function, no matter which way you used to build the model.

In [ ]:
sample_data = np.random.uniform(size=[1, 28, 28, 3])

print(sequential_model(sample_data))
print(functional_model(sample_data))
print(subclass_model(sample_data))

[[0.72899467 0.27100533]]
[[0.18588641 0.8141136 ]]
[[0.8943892 0.1056108]]


## Models can be sliced

In [ ]:
sliced_model = keras.Model(
    inputs=functional_model.inputs,
    outputs=functional_model.get_layer("conv2d_1").output,
)
sliced_model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 28, 28, 3)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_1 (Conv2D)               │ (None, 27, 27, 8)         │        104 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 104 (416.00 B)

 Trainable params: 104 (416.00 B)

 Non-trainable params: 0 (0.00 B)

# Train the model

Now we have defined the model. For actual training, we need the following pieces:
- Data
- Loss/Target function
- Optimizer

In [ ]:
# Prepare data for a 2-class classification.
data = np.random.uniform(size=[20, 28, 28, 3])
label = np.random.randint(2, size=20)

In [5]:
# Let's use functional model defined above.
model = functional_model

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(0.001),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [ ]:
model.fit(
    data,
    label,
    batch_size=5,
    epochs=2,
)

Epoch 1/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - loss: 0.7914 - sparse_categorical_accuracy: 0.4267
Epoch 2/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.7838 - sparse_categorical_accuracy: 0.4000 


# Eager mode and graph mode (demo with TF syntax)

Debug in eager mode, put actual job in graph mode. Read more [here](https://www.tensorflow.org/guide/basics#graphs_and_tffunction).

In [ ]:
# Eager mode
print(model(data[0:1, ...]))

# Graph mode
print(model.predict(data[0:1, ...]))

tf.Tensor([[0.03376885 0.9662311 ]], shape=(1, 2), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step
[[0.03376885 0.9662311 ]]


In [ ]:
@tf.function(jit_compile=True)
def forward_pass(x):
    outputs = model(x)
    print(outputs)
    return outputs

print(forward_pass(data[0:1, ...]))

Tensor("functional_2_1/softmax_1_1/Softmax:0", shape=(1, 2), dtype=float32)
tf.Tensor([[0.03376885 0.9662311 ]], shape=(1, 2), dtype=float32)


In [ ]:
# You can also run `fit`, `predict` in eager mode.
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(0.001),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    run_eagerly=True
)

# Run `predict` in eager mode by setting `run_eagerly` above.
print(model.predict(data[0:1, ...]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
[[0.03376885 0.9662311 ]]


# Write your custom training loop (demo with TF syntax)

We encourage using `fit` for training/finetuning as much as possible, but it's still important to write you custom training loop. For example, if you want to debug the training loop, or have a better control over each step, you can use custom training loop.

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((data, label))
train_data = train_data.batch(5).cache().prefetch(tf.data.AUTOTUNE)

In [ ]:
loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.Adam(0.001)
metric = keras.metrics.SparseCategoricalAccuracy()

In [ ]:
@tf.function
def train_step(data):
    x, y = data
    with tf.GradientTape() as tape:
        outputs = model(x)
        loss = loss_fn(y, outputs)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    metric.update_state(y, outputs)

    return loss


for i, data in enumerate(train_data):
    loss = train_step(data)
    print(f"Batch {i}, loss: {loss:.3f}, accuracy: {metric.result():.3f}")

NotImplementedError: ignored

# Save your model

In [ ]:
# Save the weights.
model.save_weights("dummy.weights.h5")

In [ ]:
sequential_model.load_weights("dummy.weights.h5")

In [ ]:
tf.reduce_sum(sequential_model.get_layer("conv2d").kernel - model.get_layer("conv2d_1").kernel)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

# Use MLflow to track your experiments

MLflow is a handy tool for managing the lifecycle of your ML experiments. Here we show how to use it for experiment tracking purposes.

If you have not, please register an account of [Databricks community edition](https://www.databricks.com/try-databricks#account). It should take no longer than 1min to register. Databricks CE (community edition) is a free platform for users to try out Databricks features. For this guide, we need the ML experiment dashboard for us to track our training progress.

In [4]:
!pip install -q git+https://github.com/chenmoneygithub/mlflow.git@keras-core

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00


After you have sucessfully registered an account, all you need to do is to run the command below to connect from Google Colab to your Databricks account. You will need to enter following information at prompt:
- **Databricks Host**: https://community.cloud.databricks.com/
- **Username**: your signed up email
- **Password**: your password

In [3]:
!databricks configure

Databricks Host (should begin with https://): https://community.cloud.databricks.com/
Username: qianchen94era@gmail.com
Password: 
Repeat for confirmation: 
Error: The two entered values do not match.
Password: 
Repeat for confirmation: 


In [6]:
import mlflow

mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/2023-ghc-keras-demo")

# Prepare data for a 2-class classification.
data = np.random.uniform(size=[200, 28, 28, 3])
label = np.random.randint(2, size=200)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(0.001),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)


with mlflow.start_run() as run:
    model.fit(
        data,
        label,
        batch_size=2,
        epochs=10,
        callbacks=[mlflow.keras_core.MLflowCallback(run, False, 50)]
    )

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.8228 - sparse_categorical_accuracy: 0.4561
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6202 - sparse_categorical_accuracy: 0.6119
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6283 - sparse_categorical_accuracy: 0.6514
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5203 - sparse_categorical_accuracy: 0.8422
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4539 - sparse_categorical_accuracy: 0.9188
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3953 - sparse_categorical_accuracy: 0.9644
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3440 - sparse_categorical_accuracy: 0.9813
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2825 - sparse_categorical_accuracy: 0.9901
Epoch 9/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2395 - sparse_categorical_accuracy: 0.9944
Epoch 10/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - 

# Instruction Finetuning a Generative Model

We will show how to finetune a GPT2 model with Dolly dataset. We use GPT2 for demonstration purpose, users can switch it to other KerasNLP models.

In [6]:
!pip install -q keras-nlp datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.5/573.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 26.4 MB/s eta 0:00:00


In [7]:
import keras_nlp

gpt2_causal_lm = keras_nlp.models.GPT2CausalLM.from_preset("gpt2_base_en")
gpt2_causal_lm.generate("I think basketball is fun because", max_length=50)

"I think basketball is fun because it's about fun and you can get to know people who you can relate to, and you can get to see what's going on around you, so it's a great way to learn and to learn and to"

In [8]:
# Load Dolly2 dataset from HuggingFace
from datasets import load_dataset

dolly = load_dataset("databricks/databricks-dolly-15k", split="train")

Format data with a prompt template.

In [9]:
prompt_template = {
    "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n{response}",
    "prompt_no_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n{response}",
}

In [10]:
def format_text(data):
    if len(data["context"]) == 0:
        full_text = prompt_template["prompt_no_input"].format(
            instruction=data["instruction"],
            input=data["context"],
            response=data["response"],
        )
    else:
        full_text = prompt_template["prompt_input"].format(
            instruction=data["instruction"],
            input=data["context"],
            response=data["response"],
        )

    return full_text

In [11]:
dolly_text = []
for data in dolly:
    dolly_text.append(format_text(data))

In [12]:
dolly_tf_data = tf.data.Dataset.from_tensor_slices(dolly_text)
dolly_tf_data = dolly_tf_data.batch(4).cache().prefetch(tf.data.AUTOTUNE)

In [13]:
num_epochs = 2
lr = keras.optimizers.schedules.PolynomialDecay(
    5e-5,
    decay_steps=dolly_tf_data.cardinality() * num_epochs,
    end_learning_rate=0.0,
)
gpt2_causal_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.AdamW(lr, weight_decay=0.001),
    metrics=[keras.metrics.SparseCategoricalAccuracy(),]
)

Train the model and log to MLflow.

In [14]:
import mlflow

with mlflow.start_run() as run:
    gpt2_causal_lm.fit(
        dolly_tf_data.take(100),
        epochs=num_epochs,
        callbacks=[
            mlflow.keras_core.MLflowCallback(
                run,
                log_every_epoch=False,
                log_every_n_steps=20,
            )],
    )

Epoch 1/2
100/100 ━━━━━━━━━━━━━━━━━━━━ 213s 1s/step - loss: 0.5445 - sparse_categorical_accuracy: 0.0932
Epoch 2/2
100/100 ━━━━━━━━━━━━━━━━━━━━ 117s 1s/step - loss: 0.4398 - sparse_categorical_accuracy: 0.1072


In [15]:
gpt2_causal_lm.generate("I think basketball is fun because", max_length=50)

"I think basketball is fun because it's fun, but it's also hard to learn from. You can't really learn how to play basketball from the outside. You have to learn how to play the game. You have to learn how to play"